# Flatten gridded data for trend fitting

In [1]:
# import sys; sys.path.append('/rds/general/user/cb2714/home/wwa'); from wwa import *
import xarray as xr, regionmask

In [2]:
# load daily data
da = xr.open_dataset("era5_tmax_daily_extended_-125--75E_9-37N_su.nc").tmax

In [3]:
# mask land only
rm = regionmask.defined_regions.natural_earth_v5_0_0.land_10.mask(da.lon, da.lat)
da = da.where(rm == 0)

# resample to annual
da = da.rolling(time = 5).mean().resample(time = "AS").max()

# relabel dates as years
da = da.assign_coords(time = da.time.dt.year).rename(time = "year")

# save the map for easier reconstruction later
da.mean("year").to_netcdf("map-tmplt_tx5x_era5.nc")

# flatten & convert to data.frame
df = da.stack(xy = ["lat", "lon"]).dropna("xy", "all").to_pandas()

# save data.frame as .csv (split into chunks if really large)
ncols = 2500
if df.shape[1] > ncols:
    for i in range(int(np.ceil(df.shape[1] / ncols))):
        df.iloc[:,slice(i*ncols,(i+1)*ncols)].to_csv("flattened/tx5x-flattened_era5_"+str(i+1).rjust(2,"0")+".csv")
else:
    df.to_csv("flattened/tx5x-flattened_era5.csv")

ERROR 1: PROJ: proj_create_from_database: Open of /rds/general/user/cb2714/home/anaconda3/envs/wwa/share/proj failed
